In [1]:
import hight_res as hr

/Users/salvatoremariocarota/opt/anaconda3/envs/multi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/salvatoremariocarota/opt/anaconda3/envs/multi/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


: 

In [ ]:
from PyQt6.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QLabel, QFileDialog, QHBoxLayout, QMainWindow
from PyQt6.QtMultimedia import QMediaPlayer, QAudioOutput
from PyQt6.QtMultimediaWidgets import QVideoWidget
from PyQt6.QtCore import QUrl
import cv2
import os

class VideoWindow(QMainWindow):
    def __init__(self, video_path):
        super().__init__()
        self.setWindowTitle("Riproduzione Video")
        self.setGeometry(500, 100, 800, 600)

        self.video_widget = QVideoWidget()
        self.setCentralWidget(self.video_widget)

        self.media_player = QMediaPlayer()
        self.audio_output = QAudioOutput()
        self.media_player.setAudioOutput(self.audio_output)
        self.media_player.setVideoOutput(self.video_widget)

        self.media_player.setSource(QUrl.fromLocalFile(video_path))
        self.media_player.play()

        # Controlli di riproduzione
        self.control_layout = QHBoxLayout()
        self.play_button = QPushButton("Play")
        self.pause_button = QPushButton("Pause")

        self.play_button.clicked.connect(self.media_player.play)
        self.pause_button.clicked.connect(self.media_player.pause)

        self.control_layout.addWidget(self.play_button)
        self.control_layout.addWidget(self.pause_button)

        self.layout = QVBoxLayout()
        self.layout.addWidget(self.video_widget)
        self.layout.addLayout(self.control_layout)

        widget = QWidget()
        widget.setLayout(self.layout)
        self.setCentralWidget(widget)

class MyApp(QWidget):
    def __init__(self):
        super().__init__()
        self.setGeometry(50, 100, 400, 300)

        self.layout = QVBoxLayout()
        self.label = QLabel("Premi un pulsante")
        self.layout.addWidget(self.label)

        # Pulsante per importare il video
        self.button1 = QPushButton("Importa Video")
        self.button1.clicked.connect(self.import_video)
        self.layout.addWidget(self.button1)

        # Pulsante per dividere in frame (inizialmente nascosto)
        self.button2 = QPushButton("Dividi in Frame")
        self.button2.clicked.connect(self.split_frames)
        self.button2.setVisible(False)
        self.layout.addWidget(self.button2)
        
        # Pulsante per ulteriori operazioni dopo la divisione (inizialmente nascosto)
        self.button3 = QPushButton("High Res")
        self.button3.clicked.connect(self.high_res)
        self.button3.setVisible(False)
        self.layout.addWidget(self.button3)

        self.video_path = None
        
        self.setLayout(self.layout)
        self.setWindowTitle("Gestione Video")

    def import_video(self):
        self.button3.setVisible(False)
        file_dialog = QFileDialog()
        file_path, _ = file_dialog.getOpenFileName(self, "Seleziona un video", "", "Video Files (*.mp4 *.avi *.mov)")
        
        if file_path:
            self.video_path = file_path
            self.label.setText(f"Video importato: {os.path.basename(file_path)}")
            self.button2.setVisible(True)  # Mostra il pulsante per dividere in frame
            self.open_video_window()
    
    def open_video_window(self):
        if self.video_path:
            self.video_window = VideoWindow(self.video_path)
            self.video_window.show()
    
    def split_frames(self):
        if not self.video_path:
            self.label.setText("Nessun video selezionato!")
            return
        
        cap = cv2.VideoCapture(self.video_path)
        frame_count = 0
        output_folder = "frames"
        os.makedirs(output_folder, exist_ok=True)
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.png")
            cv2.imwrite(frame_filename, frame)
            frame_count += 1
        
        cap.release()
        self.label.setText(f"Video diviso in {frame_count} frame salvati in '{output_folder}'")
        self.button2.setVisible(False)
        self.button3.setVisible(True)  
  
    
    def high_res(self):
        self.label.setText("Funzione High Res in esecuzione...")
        hr.hight_res(self.label)
        self.label.setText("Fine operazione")

if __name__ == "__main__":
    app = QApplication([])
    window = MyApp()
    window.show()
    app.exec()


qt.multimedia.ffmpeg: Using Qt multimedia with FFmpeg version 7.1 LGPL version 2.1 or later
qt.multimedia.ffmpeg: Available HW decoding frameworks:
qt.multimedia.ffmpeg:      videotoolbox
qt.multimedia.ffmpeg: Available HW encoding frameworks:
qt.multimedia.ffmpeg:      videotoolbox


loading model from model_zoo/swinir/001_classicalSR_DIV2K_s48w8_SwinIR-M_x8.pth


/Users/salvatoremariocarota/opt/anaconda3/envs/multi/lib/python3.9/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
